# Stratify to perform linear modeling on certain data

In [1]:
import pathlib
import pandas as pd

from sklearn.linear_model import LinearRegression

from pycytominer import feature_select
from pycytominer.cyto_utils import infer_cp_features

In [2]:
# Define inputs and outputs
plate = "localhost230405150001"  # Focusing on plate 3
file_suffix = "_sc_norm_fs_cellprofiler.csv.gz"

data_dir = pathlib.Path("../../../3.process_cfret_features/data/")

data_df = pd.read_csv(pathlib.Path(data_dir, f"{plate}{file_suffix}"))

output_dir = pathlib.Path("results")
output_cp_file = pathlib.Path(output_dir, f"{plate}_linear_model_DMSO_failing_healthy.tsv")

print(data_df.shape)
data_df.head()


(26992, 710)


/var/folders/9x/_3wvhybd07351zv_2xdxb3kw0000gn/T/ipykernel_95963/3129892645.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(pathlib.Path(data_dir, f"{plate}{file_suffix}"))


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,...,Nuclei_Texture_InfoMeas2_PM_3_01_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Actin_3_02_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_01_256,Nuclei_Texture_SumVariance_PM_3_01_256
0,B,2,9,failing,rejected,DMSO,1,localhost230405150001,B02,1,...,0.191918,-0.032872,0.292288,-0.604487,0.888165,0.429366,0.399530,-0.366829,-0.258781,-0.310718
1,B,2,9,failing,rejected,DMSO,1,localhost230405150001,B02,2,...,0.608292,0.290130,-0.069668,-0.411109,0.182782,0.447620,0.350265,-0.380608,2.405688,-0.190068
2,B,2,9,failing,rejected,DMSO,1,localhost230405150001,B02,3,...,-0.140377,-0.314924,-0.217099,-0.612188,0.524718,0.798054,0.552916,-0.401958,-0.232218,-0.316777
3,B,2,9,failing,rejected,DMSO,1,localhost230405150001,B02,4,...,1.021672,0.813690,0.616922,-0.400131,0.260481,0.714237,0.411960,-0.367868,-0.152203,-0.217675
4,B,2,9,failing,rejected,DMSO,1,localhost230405150001,B02,5,...,-0.170594,0.078071,0.048193,0.768125,-0.580192,0.982025,0.979740,-0.395945,0.034628,-0.348407


## Stratify data

In [3]:
# Filter by failing hearts and specific treatments
specific_type = ["DMSO"]
specific_cell_types = ["failing", "healthy"]

filtered_df = data_df[
    (data_df['Metadata_treatment'].isin(specific_type)) &
    (data_df['Metadata_cell_type'].isin(specific_cell_types))
]

# Drop NA columns
cp_df = feature_select(
    filtered_df,
    operation="drop_na_columns",
    na_cutoff=0
)

# Count number of cells per well and add to dataframe as metadata
cell_count_df = pd.DataFrame(
    cp_df.groupby("Metadata_Well").count()["Metadata_treatment"]
).reset_index()
cell_count_df.columns = ["Metadata_Well", "Metadata_cell_count_per_well"]
cp_df = cell_count_df.merge(cp_df, on=["Metadata_Well"])

# Define CellProfiler features
cp_features = infer_cp_features(cp_df)

print(f"We are testing {len(cp_features)} CellProfiler features")
print(cp_df.shape)
cp_df.head()


We are testing 555 CellProfiler features
(13200, 569)


,Metadata_Well,Metadata_cell_count_per_well,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_ImageNumber,Metadata_Plate,...,Nuclei_Texture_InfoMeas2_PM_3_01_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Actin_3_02_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_01_256,Nuclei_Texture_SumVariance_PM_3_01_256
0,B02,564,B,2,9,failing,rejected,DMSO,1,localhost230405150001,...,0.191918,-0.032872,0.292288,-0.604487,0.888165,0.429366,0.399530,-0.366829,-0.258781,-0.310718
1,B02,564,B,2,9,failing,rejected,DMSO,1,localhost230405150001,...,0.608292,0.290130,-0.069668,-0.411109,0.182782,0.447620,0.350265,-0.380608,2.405688,-0.190068
2,B02,564,B,2,9,failing,rejected,DMSO,1,localhost230405150001,...,-0.140377,-0.314924,-0.217099,-0.612188,0.524718,0.798054,0.552916,-0.401958,-0.232218,-0.316777
3,B02,564,B,2,9,failing,rejected,DMSO,1,localhost230405150001,...,1.021672,0.813690,0.616922,-0.400131,0.260481,0.714237,0.411960,-0.367868,-0.152203,-0.217675
4,B02,564,B,2,9,failing,rejected,DMSO,1,localhost230405150001,...,-0.170594,0.078071,0.048193,0.768125,-0.580192,0.982025,0.979740,-0.395945,0.034628,-0.348407


## Fit linear model

In [4]:
# Setup linear modeling framework -> in plate 3 we are looking at the treatments or cell type
variables = ["Metadata_cell_count_per_well", "Metadata_cell_type"]
X = cp_df.loc[:, variables]

print(X.shape)
X.head()

(13200, 2)


,Metadata_cell_count_per_well,Metadata_cell_type
0,564,failing
1,564,failing
2,564,failing
3,564,failing
4,564,failing


In [5]:
# Assuming cp_df is your DataFrame
variables = ["Metadata_cell_count_per_well", "Metadata_cell_type"]
treatments_to_select = ["failing", "healthy"]

# Select rows with specific treatment values
selected_rows = X[X["Metadata_cell_type"].isin(treatments_to_select)]

# Create dummy variables
dummies = pd.get_dummies(selected_rows["Metadata_cell_type"])

# Concatenate dummies with the selected rows DataFrame
X = pd.concat([selected_rows, dummies], axis=1)

# Drop the original treatment column
X.drop("Metadata_cell_type", axis=1, inplace=True)

print(X.shape)
X.head()

(13200, 3)


,Metadata_cell_count_per_well,failing,healthy
0,564,1,0
1,564,1,0
2,564,1,0
3,564,1,0
4,564,1,0


In [6]:
# Fit linear model for each feature
lm_results = []
for cp_feature in cp_features:
    # Create a boolean mask to filter rows with the specified treatments
    mask = cp_df["Metadata_cell_type"].isin(treatments_to_select)

    # Apply the mask to Subset CP data to each individual feature (univariate test)
    cp_subset_df = cp_df.loc[mask, cp_feature]

    # Fit linear model
    lm = LinearRegression(fit_intercept=True)
    lm_result = lm.fit(X=X, y=cp_subset_df)
    
    # Extract Beta coefficients
    # (contribution of feature to X covariates)
    coef = lm_result.coef_
    
    # Estimate fit (R^2)
    r2_score = lm.score(X=X, y=cp_subset_df)
    
    # Add results to a growing list
    lm_results.append([cp_feature, r2_score] + list(coef))

# Convert results to a pandas DataFrame
lm_results = pd.DataFrame(
    lm_results,
    columns=["feature", "r2_score", "cell_count_coef", "failing_coef", "healthy_coef"]
)

# Output file
lm_results.to_csv(output_cp_file, sep="\t", index=False)

print(lm_results.shape)
lm_results.head()

(555, 5)


,feature,r2_score,cell_count_coef,failing_coef,healthy_coef
0,Cytoplasm_AreaShape_Area,0.027071,-0.001083,0.296105,-0.296105
1,Cytoplasm_AreaShape_Compactness,0.000216,-0.000091,0.025964,-0.025964
2,Cytoplasm_AreaShape_Eccentricity,0.007732,-0.000212,-0.076832,0.076832
3,Cytoplasm_AreaShape_FormFactor,0.000045,-0.000019,0.012692,-0.012692
4,Cytoplasm_AreaShape_MajorAxisLength,0.021964,-0.000905,0.112036,-0.112036
